# WORDCLOUD

## Overview
This function generates a word cloud image from provided text data, visualizing the most frequent words in a visually engaging format. The resulting chart is returned as a PNG image encoded as a base64 string, suitable for embedding in Excel or web apps. This is useful for quickly summarizing key themes or topics from survey responses, customer feedback, or any large collection of text. The function uses the `wordcloud` and `matplotlib` Python packages.

## Usage
To insert the image in a viewable manner in Excel, use the function dialog and select `Insert result, not formula`. Using the custom function below will only return the base64 string.

```excel
=WORDCLOUD(text, [max_words], [background_color], [colormap])
```

## Arguments
| Argument         | Type                | Required/Optional | Description                                                                 | Example                |
|------------------|---------------------|-------------------|-----------------------------------------------------------------------------|------------------------|
| text             | list[list[string]]  | Required          | The text data to analyze. Can be a range of cells or a 2D list of strings.  | [["Great service"]]   |
| max_words        | float               | Optional          | Maximum number of words to display in the word cloud. Default is 100.       | 50                     |
| background_color | string              | Optional          | Background color of the image. Default is "white".                          | "black"               |
| colormap         | string              | Optional          | Matplotlib colormap for word colors. Default is "viridis".                  | "plasma"              |

## Returns
| Return Value | Type   | Description                                                      | Example         |
|--------------|--------|------------------------------------------------------------------|-----------------|
| Image        | string | PNG image as a base64 string for embedding.                      | "data:image/png;base64,..." |
| Error        | string | Error message if an invalid input or other error occurs.          | "Invalid input: text must be a list of strings." |

## Limitations
- Only supports text data as 2D lists or ranges.
- Large text datasets may result in slow rendering or large images.
- Requires the `wordcloud` and `matplotlib` packages in the Python environment.

## Examples

### Customer Feedback Word Cloud
**Business Context:** Summarize the most common words from customer feedback to identify key themes.

Sample Input Table:
|   |   A              |   B            |
|---|------------------|----------------|
| 1 | Great service    | Fast delivery  |
| 2 | Excellent support| Great service  |

Excel Formula:
```excel
=WORDCLOUD(A1:B2, 10, "white", "viridis")
```
*This will generate a word cloud from the feedback in cells A1:B2, showing the 10 most frequent words on a white background.*

### Survey Results Visualization
**Business Context:** Visualize the main topics from open-ended survey responses for a team meeting.

Sample Input Table:
|   |   A                      |
|---|--------------------------|
| 1 | Easy to use              |
| 2 | User friendly interface  |
| 3 | Easy to use              |

Excel Formula:
```excel
=WORDCLOUD(A1:A3, 8, "black", "plasma")
```
*This will create a word cloud from survey responses in A1:A3, displaying the 8 most common words with a black background and a plasma color scheme.*

In [ ]:
options = {"insert_only":True}  # Inserts image instead of formula that returns a data URL string.

import micropip
await micropip.install('wordcloud')

import matplotlib
matplotlib.use('Agg')
import base64
from io import BytesIO
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def wordcloud(text, max_words=None, background_color=None, colormap=None):
    """
    Generates a word cloud image from provided text data and returns a PNG image as a base64 string.

    Args:
        text: 2D list of strings containing the text data to analyze.
        max_words: Optional; maximum number of words to display in the word cloud. Default is 100.
        background_color: Optional; background color of the image. Default is "white".
        colormap: Optional; matplotlib colormap for word colors. Default is "viridis".

    Returns:
        PNG image as a base64 string for embedding, or an error message string if input is invalid.
    """
    try:
        if not isinstance(text, list) or not all(isinstance(row, list) for row in text):
            return "Invalid input: text must be a 2D list of strings."
        flat_text = []
        for row in text:
            for cell in row:
                if not isinstance(cell, str):
                    return "Invalid input: all elements in text must be strings."
                flat_text.append(cell)
        joined_text = " ".join(flat_text)
        if not joined_text.strip():
            return "Invalid input: text is empty."
        if max_words is None:
            max_words = 100
        else:
            try:
                max_words = int(max_words)
            except Exception:
                return "Invalid input: max_words must be a number."
        if background_color is None:
            background_color = "white"
        if colormap is None:
            colormap = "viridis"
        wc = WordCloud(width=800, height=400, max_words=max_words, background_color=background_color, colormap=colormap)
        wc.generate(joined_text)
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.imshow(wc, interpolation='bilinear')
        ax.axis('off')
        buf = BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
        plt.close(fig)
        buf.seek(0)
        img_base64 = base64.b64encode(buf.read()).decode('utf-8')
        return f"data:image/png;base64,{img_base64}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_wordcloud_success():
    # Customer feedback example
    text = [["Great service", "Fast delivery"], ["Excellent support", "Great service"]]
    result = wordcloud(text, 10, "white", "viridis")
    assert isinstance(result, str)
    assert result.startswith("data:image/png;base64,")
    assert len(result) > 30

def test_wordcloud_survey():
    # Survey results example
    text = [["Easy to use"], ["User friendly interface"], ["Easy to use"]]
    result = wordcloud(text, 8, "black", "plasma")
    assert isinstance(result, str)
    assert result.startswith("data:image/png;base64,")
    assert len(result) > 30

def test_wordcloud_empty():
    # Empty text
    text = [[]]
    result = wordcloud(text, 10)
    assert isinstance(result, str)
    assert "empty" in result or "Invalid" in result

def test_wordcloud_invalid_type():
    # Not a 2D list
    text = "Great service"
    result = wordcloud(text, 10)
    assert isinstance(result, str)
    assert "Invalid" in result

def test_wordcloud_non_string():
    # Non-string element
    text = [[123, "Good"]]
    result = wordcloud(text, 10)
    assert isinstance(result, str)
    assert "Invalid" in result

def test_wordcloud_invalid_max_words():
    # max_words not a number
    text = [["Great service"]]
    result = wordcloud(text, "ten")
    assert isinstance(result, str)
    assert "Invalid" in result

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    # Customer Feedback Word Cloud example
    [
        [["Great service"], ["Fast delivery"], ["Excellent support"]],
        10,
        "white",
        "viridis"
    ],
    # Survey Results Visualization example
    [
        [["Easy to use"], ["User friendly interface"], ["Easy to use"]],
        8,
        "black",
        "plasma"
    ]
]

def render_wordcloud_html(text, max_words, background_color, colormap):
    result = wordcloud(text, max_words, background_color, colormap)
    if isinstance(result, str) and result.startswith("data:image/png;base64,"):
        # Return an HTML <img> tag with the data URL
        return f'<img src="{result}" alt="Word Cloud" style="max-width:100%;height:auto;" />'
    return f'<div style="color:red;">{result}</div>'

demo = gr.Interface(
    fn=render_wordcloud_html,
    inputs=[
        gr.Dataframe(headers=["Text"], show_label=False, row_count=3, col_count=1, type="array", value=[["Great service"], ["Fast delivery"]]),
        gr.Number(label="Max Words", value=100),
        gr.Textbox(label="Background Color", value="white"),
        gr.Textbox(label="Colormap", value="viridis")
    ],
    outputs=gr.HTML(label="Word Cloud Image"),
    examples=examples,
    description="Generate a word cloud from text data and return a PNG image as an HTML <img> tag.",
    flagging_mode='never',
)
demo.launch()